In [88]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col}
import org.apache.spark.ml.classification.{RandomForestClassifier, GBTClassifier}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.ml.feature.{CountVectorizer, StringIndexer, VectorAssembler, StandardScaler, MinMaxScaler}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator, CrossValidatorModel}
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}
import org.apache.spark.sql.functions.lit
import org.apache.spark.ml.feature.OneHotEncoder

import org.apache.spark.mllib.linalg.{Vector=>OldVector}
imp...


In [3]:
val spark = SparkSession.builder.appName("MBorchashvili session").getOrCreate()

spark = org.apache.spark.sql.SparkSession@3bc3e7f3


org.apache.spark.sql.SparkSession@3bc3e7f3

In [95]:
spark

lastException: Throwable = null


org.apache.spark.sql.SparkSession@3bc3e7f3

In [96]:
var trainDf = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("/labs/slaba05/lab05_train.csv")

trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 _c0                         | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M       

trainDf = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [97]:
val testDfData = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("/labs/slaba05/lab05_test.csv")
    .na.fill(0)
    .withColumn("TEST", lit(""))

testDfData = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [98]:
val allDfData = trainDf.unionAll(testDfData)

allDfData = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [99]:
val categoriesCols = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)

categoriesCols = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

# Adding Categorical Features

In [100]:
val indexer_CLNT_TRUST_RELATION = new StringIndexer()
      .setInputCol("CLNT_TRUST_RELATION")
      .setOutputCol("CLNT_TRUST_RELATION_index")
      .setHandleInvalid("keep") //skip keep error

val indexer_APP_MARITAL_STATUS = new StringIndexer()
      .setInputCol("APP_MARITAL_STATUS")
      .setOutputCol("APP_MARITAL_STATUS_index")
      .setHandleInvalid("keep") //skip keep error

val indexer_APP_KIND_OF_PROP_HABITATION = new StringIndexer()
      .setInputCol("APP_KIND_OF_PROP_HABITATION")
      .setOutputCol("APP_KIND_OF_PROP_HABITATION_index")
      .setHandleInvalid("keep") //skip keep error

val indexer_CLNT_JOB_POSITION_TYPE = new StringIndexer()
      .setInputCol("CLNT_JOB_POSITION_TYPE")
      .setOutputCol("CLNT_JOB_POSITION_TYPE_index")
      .setHandleInvalid("keep") //skip keep error

val indexer_CLNT_JOB_POSITION = new StringIndexer()
      .setInputCol("CLNT_JOB_POSITION")
      .setOutputCol("CLNT_JOB_POSITION_index")
      .setHandleInvalid("keep") //skip keep error

val indexer_APP_DRIVING_LICENSE = new StringIndexer()
      .setInputCol("APP_DRIVING_LICENSE")
      .setOutputCol("APP_DRIVING_LICENSE_index")
      .setHandleInvalid("keep") //skip keep error

val indexer_APP_EDUCATION = new StringIndexer()
      .setInputCol("APP_EDUCATION")
      .setOutputCol("APP_EDUCATION_index")
      .setHandleInvalid("keep") //skip keep error

val indexer_APP_TRAVEL_PASS = new StringIndexer()
      .setInputCol("APP_TRAVEL_PASS")
      .setOutputCol("APP_TRAVEL_PASS_index")
      .setHandleInvalid("keep") //skip keep error


val indexer_APP_CAR = new StringIndexer()
      .setInputCol("APP_CAR")
      .setOutputCol("APP_CAR_index")
      .setHandleInvalid("keep") //skip keep error


val indexer_APP_POSITION_TYPE = new StringIndexer()
      .setInputCol("APP_POSITION_TYPE")
      .setOutputCol("APP_POSITION_TYPE_index")
      .setHandleInvalid("keep") //skip keep error


val indexer_APP_EMP_TYPE = new StringIndexer()
      .setInputCol("APP_EMP_TYPE")
      .setOutputCol("APP_EMP_TYPE_index")
      .setHandleInvalid("keep") //skip keep error


val indexer_APP_COMP_TYPE = new StringIndexer()
      .setInputCol("APP_COMP_TYPE")
      .setOutputCol("APP_COMP_TYPE_index")
      .setHandleInvalid("keep") //skip keep error


val indexer_PACK = new StringIndexer()
      .setInputCol("PACK")
      .setOutputCol("PACK_index")
      .setHandleInvalid("keep") //skip keep error



import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.{OneHotEncoderEstimator, OneHotEncoderModel}

val transformationPipeline = new Pipeline()
    .setStages(Array(indexer_CLNT_TRUST_RELATION, indexer_APP_MARITAL_STATUS, indexer_APP_KIND_OF_PROP_HABITATION,
                    indexer_CLNT_JOB_POSITION_TYPE, indexer_CLNT_JOB_POSITION, indexer_APP_DRIVING_LICENSE,
                    indexer_APP_EDUCATION, indexer_APP_TRAVEL_PASS, indexer_APP_CAR, indexer_APP_POSITION_TYPE,
                    indexer_APP_EMP_TYPE, indexer_APP_COMP_TYPE, indexer_PACK))

val train_df_w_labels = transformationPipeline.fit(allDfData).transform(trainDf)
val test_df_w_labels = transformationPipeline.fit(allDfData).transform(testDfData)

indexer_CLNT_TRUST_RELATION = strIdx_f6fc139eed62
indexer_APP_MARITAL_STATUS = strIdx_8f83b47a97fd
indexer_APP_KIND_OF_PROP_HABITATION = strIdx_60e57ce45ce7
indexer_CLNT_JOB_POSITION_TYPE = strIdx_4e0e3cba49d5
indexer_CLNT_JOB_POSITION = strIdx_b27b68611d5e
indexer_APP_DRIVING_LICENSE = strIdx_197043f41c87
indexer_APP_EDUCATION = strIdx_b99870d91df1
indexer_APP_TRAVEL_PASS = strIdx_cba9b821ab36


indexer_APP_CAR: org.apache.spark.ml.feature.StringInde...


strIdx_cba9b821ab36

In [101]:
val train_df_w_labels_2 = train_df_w_labels.drop(categoriesCols:_*).na.fill(0)
val test_df_w_labels_2 = test_df_w_labels.drop(categoriesCols:_*).drop("TARGET").na.fill(0)

train_df_w_labels_2 = [_c0: int, ID: int ... 115 more fields]
test_df_w_labels_2 = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [102]:
val Array(trainDfTrain, trainDfTest) = train_df_w_labels_2.randomSplit(Array(0.8, 0.2))

trainDfTrain = [_c0: int, ID: int ... 115 more fields]
trainDfTest = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [103]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(trainDfTrain.drop("TARGET").columns)
    .setOutputCol("features")

vectorAssembler = vecAssembler_3750ca12f584


vecAssembler_3750ca12f584

In [104]:
val trainDfCleaned = vectorAssembler.transform(trainDfTrain).cache

trainDfCleaned = [_c0: int, ID: int ... 116 more fields]


[_c0: int, ID: int ... 116 more fields]

In [105]:
trainDfCleaned.show(1, 0, true)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 _c0                               | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                
 ID                                | 146841               

In [106]:
val GBTModel = new GBTClassifier()
  .setLabelCol("TARGET")
  .setFeaturesCol("features")
  .setFeatureSubsetStrategy("auto").fit(trainDfCleaned)

GBTModel = GBTClassificationModel (uid=gbtc_89e8b7e926e2) with 20 trees


GBTClassificationModel (uid=gbtc_89e8b7e926e2) with 20 trees

In [107]:
val predictions = GBTModel.transform(
    vectorAssembler.transform(trainDfTest)
).cache

predictions
    .select(
        "TARGET",
        "features",
        "rawPrediction",
        "probability",
        "prediction"
    )
    .show(5)

+------+--------------------+--------------------+--------------------+----------+
|TARGET|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|(116,[0,1,37,38,5...|[0.43606581532409...|[0.70518905232983...|       0.0|
|     0|(116,[0,1,5,11,13...|[1.49356329379248...|[0.95198914894640...|       0.0|
|     0|(116,[0,1,3,9,10,...|[1.48741024992158...|[0.95142355040300...|       0.0|
|     0|(116,[0,1,3,5,9,1...|[1.19245001818081...|[0.91566858474432...|       0.0|
|     0|(116,[0,1,3,9,10,...|[1.16202152811614...|[0.91084879584933...|       0.0|
+------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



predictions = [_c0: int, ID: int ... 119 more fields]


[_c0: int, ID: int ... 119 more fields]

In [108]:
predictions.printSchema()

root
 |-- _c0: integer (nullable = false)
 |-- ID: integer (nullable = false)
 |-- CR_PROD_CNT_IL: integer (nullable = false)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = false)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = false)
 |-- APP_REGISTR_RGN_CODE: double (nullable = false)
 |-- PRC_ACCEPTS_A_POS: double (nullable = false)
 |-- PRC_ACCEPTS_A_TK: double (nullable = false)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = false)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = false)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = false)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = false)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = false)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = false)
 |-- PRC_ACCEPTS_TK: double (nullable = false)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = false)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = false)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = false)
 |-- CNT_ACCEPTS_TK: double (nullable = false)
 |-- REST_DYNAMIC_SAVE_3M: double

In [109]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [110]:
predictions
    .select(vectorToArrayUdf(col("probability")).getItem(1))
    .show(5)

+--------------------+
| UDF(probability)[1]|
+--------------------+
| 0.29481094767016813|
| 0.04801085105359382|
|0.048576449596990146|
|   0.084331415255671|
| 0.08915120415066313|
+--------------------+
only showing top 5 rows



In [111]:
val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("TARGET")
    .setRawPredictionCol("prediction")
    .setMetricName("areaUnderROC")

val accuracy = evaluator
    .evaluate(
        predictions.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    )

evaluator = binEval_fa87886c0b70
accuracy = 0.8204363400240859


0.8204363400240859

## TEST

In [112]:
val realPredictions = GBTModel.transform(vectorAssembler.transform(test_df_w_labels_2)).cache

realPredictions = [_c0: int, ID: int ... 119 more fields]


[_c0: int, ID: int ... 119 more fields]

In [113]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .show(5)

+------+-------------------+
|    ID|         prediction|
+------+-------------------+
|519130|0.14858176063890438|
|234045|0.05097217168062762|
|401256|0.04900388559171254|
|551070|0.07142147319259684|
|367285|0.05181430255646935|
+------+-------------------+
only showing top 5 rows



In [114]:
realPredictions

[_c0: int, ID: int ... 119 more fields]

In [115]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .withColumnRenamed("ID", "id")
    .withColumnRenamed("prediction", "target")
    .coalesce(1)
    .write
    .option("header", "true")
    .mode("overwrite")
    .format("csv")
    .option("sep" ,"\t")
    .save("lab05.csv")